In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from ipywidgets import interact, widgets
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# Load the datasets
companies_df = pd.read_csv('sp500_companies.csv')
stocks_df = pd.read_csv('sp500_stocks.csv')
index_df = pd.read_csv('sp500_index.csv')

# Convert date columns to datetime
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
index_df['Date'] = pd.to_datetime(index_df['Date'])

# Clean and prepare market cap data
companies_df['Marketcap'] = pd.to_numeric(companies_df['Marketcap'], errors='coerce')

print("Data loaded successfully!")
print(f"Companies: {len(companies_df)}")
print(f"Stock records: {len(stocks_df):,}")
print(f"Index records: {len(index_df):,}")
print(f"Date range: {stocks_df['Date'].min()} to {stocks_df['Date'].max()}")

Data loaded successfully!
Companies: 502
Stock records: 1,891,536
Index records: 2,517
Date range: 2010-01-04 00:00:00 to 2024-12-20 00:00:00


In [3]:
# Function to analyze sector distribution
def analyze_sector_distribution():
    """
    Analyzes and displays sector distribution by count and market cap
    """
    print("\n" + "="*80)
    print("SECTOR DISTRIBUTION ANALYSIS")
    print("="*80 + "\n")
    
    # Count by sector
    sector_counts = companies_df['Sector'].value_counts().reset_index()
    sector_counts.columns = ['Sector', 'Number of Companies']
    
    # Market cap by sector
    sector_marketcap = companies_df.groupby('Sector')['Marketcap'].sum().reset_index()
    sector_marketcap.columns = ['Sector', 'Total Market Cap']
    sector_marketcap['Market Cap (Billions)'] = sector_marketcap['Total Market Cap'] / 1e9
    sector_marketcap = sector_marketcap.sort_values('Total Market Cap', ascending=False)
    
    # Merge the two dataframes
    sector_summary = sector_counts.merge(sector_marketcap[['Sector', 'Market Cap (Billions)']], on='Sector')
    sector_summary = sector_summary.sort_values('Market Cap (Billions)', ascending=False)
    
    # Add percentage
    total_marketcap = companies_df['Marketcap'].sum()
    sector_summary['Percent of Total'] = (sector_summary['Market Cap (Billions)'] * 1e9 / total_marketcap * 100).round(1)
    
    print("Summary by Sector:")
    display(sector_summary)
    
    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Market Cap by Sector
    sns.barplot(data=sector_summary, y='Sector', x='Market Cap (Billions)', ax=ax1)
    ax1.set_title('Market Capitalization by Sector', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Market Cap (Billions $)', fontsize=12)
    ax1.set_ylabel('Sector', fontsize=12)
    
    # Plot 2: Percentage Distribution
    sns.barplot(data=sector_summary, y='Sector', x='Percent of Total', ax=ax2)
    ax2.set_title('Percentage of Total Market Cap by Sector', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Percent of Total Market Cap (%)', fontsize=12)
    ax2.set_ylabel('Sector', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Get the top sector
    top_sector = sector_summary.iloc[0]
    
    print("\n" + "-"*80)
    print("KEY FINDINGS:")
    print("-"*80)
    print(f"Total Market Cap: ${total_marketcap/1e12:.2f} Trillion")
    print(f"\nTop Sector: {top_sector['Sector']}")
    print(f"  Market Cap: ${top_sector['Market Cap (Billions)']:.2f} Billion")
    print(f"  Percentage: {top_sector['Percent of Total']:.1f}%")
    print(f"  Number of Companies: {top_sector['Number of Companies']} out of {len(companies_df)} total")
    print(f"\n{top_sector['Sector']} companies represent {top_sector['Number of Companies']/len(companies_df)*100:.1f}% of companies")
    print(f"  but {top_sector['Percent of Total']:.1f}% of total market cap!")

In [4]:
# Function to analyze top companies
def analyze_top_companies(n=20):
    """
    Analyzes the top companies by market cap
    """
    print("\n" + "="*80)
    print(f"TOP {n} COMPANIES ANALYSIS")
    print("="*80 + "\n")
    
    # Find the right column name for company names
    if 'Security' in companies_df.columns:
        name_col = 'Security'
    elif 'Shortname' in companies_df.columns:
        name_col = 'Shortname'
    elif 'Longname' in companies_df.columns:
        name_col = 'Longname'
    elif 'Name' in companies_df.columns:
        name_col = 'Name'
    else:
        name_col = 'Symbol'
    
    # Get the columns we need
    cols_to_get = ['Symbol', name_col, 'Sector', 'Marketcap']
    cols_to_get = list(dict.fromkeys(cols_to_get))
    
    top_companies = companies_df.nlargest(n, 'Marketcap')[cols_to_get].copy()
    top_companies['Market Cap (Billions)'] = (top_companies['Marketcap'] / 1e9).round(2)
    
    # Prepare display columns
    display_cols = ['Symbol', name_col, 'Sector', 'Market Cap (Billions)']
    display_cols = list(dict.fromkeys(display_cols))
    top_companies = top_companies[display_cols]
    top_companies['Rank'] = range(1, len(top_companies) + 1)
    
    # Reorder with Rank first
    final_cols = ['Rank'] + display_cols
    top_companies = top_companies[final_cols]
    
    print(f"Top {n} Companies by Market Capitalization:")
    display(top_companies)
    
    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Top companies bar chart
    plot_data = top_companies.head(10).copy()
    sns.barplot(data=plot_data, y='Symbol', x='Market Cap (Billions)', ax=ax1, hue='Sector', dodge=False, legend=False)
    ax1.set_title('Top 10 Companies by Market Cap', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Market Cap (Billions $)', fontsize=12)
    ax1.set_ylabel('Company Symbol', fontsize=12)
    ax1.invert_yaxis()
    
    # Plot 2: Sector distribution pie chart
    sector_dist = top_companies['Sector'].value_counts()
    ax2.pie(sector_dist.values, labels=sector_dist.index, autopct='%1.1f%%', startangle=90)
    ax2.set_title(f'Sector Distribution in Top {n} Companies', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Sector distribution table
    print("\n" + "-"*80)
    print(f"Sector Distribution in Top {n}:")
    print("-"*80)
    sector_dist_df = top_companies['Sector'].value_counts().reset_index()
    sector_dist_df.columns = ['Sector', 'Count']
    sector_dist_df['Percentage'] = (sector_dist_df['Count'] / n * 100).round(1)
    display(sector_dist_df)
    
    # Print key findings
    top_sector_name = sector_dist_df.iloc[0]['Sector']
    top_sector_count = sector_dist_df.iloc[0]['Count']
    
    print("\n" + "-"*80)
    print("KEY FINDINGS:")
    print("-"*80)
    print(f"Top sector in top {n}: {top_sector_name} with {top_sector_count} companies ({top_sector_count/n*100:.1f}%)")
    
    print(f"\nTop 5 companies:")
    for idx, row in top_companies.head().iterrows():
        company_name = row[name_col] if name_col != 'Symbol' else ''
        if company_name and company_name != row['Symbol']:
            print(f"  #{row['Rank']} {row['Symbol']:6s} - {company_name:40s} ${row['Market Cap (Billions)']:8.1f}B ({row['Sector']})")
        else:
            print(f"  #{row['Rank']} {row['Symbol']:6s} ${row['Market Cap (Billions)']:8.1f}B ({row['Sector']})")

In [5]:
# Function to analyze stock performance
def analyze_performance(start_date='2015-01-01', end_date='2023-12-31'):
    """
    Analyzes stock performance comparing top sector vs other sectors
    """
    print("\n" + "="*80)
    print("PERFORMANCE COMPARISON: TOP SECTOR vs OTHERS")
    print("="*80 + "\n")
    
    # Filter data by date range
    mask = (stocks_df['Date'] >= start_date) & (stocks_df['Date'] <= end_date)
    period_stocks = stocks_df[mask].copy()
    
    # Merge with company info
    stocks_with_sector = period_stocks.merge(companies_df[['Symbol', 'Sector']], on='Symbol')
    
    # Get the top sector by market cap
    sector_marketcap = companies_df.groupby('Sector')['Marketcap'].sum().sort_values(ascending=False)
    top_sector_name = sector_marketcap.index[0]
    
    print(f"Comparing {top_sector_name} vs Other Sectors\n")
    
    # Mark top sector
    stocks_with_sector['IsTopSector'] = stocks_with_sector['Sector'] == top_sector_name
    
    # Calculate daily returns for each stock (fix for FutureWarning)
    stocks_with_sector = stocks_with_sector.sort_values(['Symbol', 'Date'])
    stocks_with_sector['Returns'] = stocks_with_sector.groupby('Symbol')['Close'].pct_change(fill_method=None)
    
    # Get average returns by sector group
    top_sector_returns = stocks_with_sector[stocks_with_sector['IsTopSector'] == True]['Returns']
    other_sectors_returns = stocks_with_sector[stocks_with_sector['IsTopSector'] == False]['Returns']
    
    # Calculate statistics
    performance_data = {
        'Metric': [
            'Average Daily Return (%)',
            'Median Daily Return (%)',
            'Std Deviation (%)',
            'Total Days',
            'Positive Days',
            'Negative Days'
        ],
        top_sector_name: [
            top_sector_returns.mean() * 100,
            top_sector_returns.median() * 100,
            top_sector_returns.std() * 100,
            len(top_sector_returns.dropna()),
            (top_sector_returns > 0).sum(),
            (top_sector_returns < 0).sum()
        ],
        'Other Sectors': [
            other_sectors_returns.mean() * 100,
            other_sectors_returns.median() * 100,
            other_sectors_returns.std() * 100,
            len(other_sectors_returns.dropna()),
            (other_sectors_returns > 0).sum(),
            (other_sectors_returns < 0).sum()
        ]
    }
    
    performance_df = pd.DataFrame(performance_data)
    performance_df[top_sector_name] = performance_df[top_sector_name].round(4)
    performance_df['Other Sectors'] = performance_df['Other Sectors'].round(4)
    
    print(f"Analysis Period: {start_date} to {end_date}\n")
    display(performance_df)
    
    # Calculate cumulative returns for visualization
    stocks_with_sector_sorted = stocks_with_sector.sort_values('Date')
    daily_avg_returns = stocks_with_sector_sorted.groupby(['Date', 'IsTopSector'])['Returns'].mean().reset_index()
    
    # Create separate series for top sector and others
    top_sector_daily = daily_avg_returns[daily_avg_returns['IsTopSector'] == True].set_index('Date')['Returns']
    other_sectors_daily = daily_avg_returns[daily_avg_returns['IsTopSector'] == False].set_index('Date')['Returns']
    
    # Calculate cumulative returns
    top_cumulative_series = (1 + top_sector_daily).cumprod()
    other_cumulative_series = (1 + other_sectors_daily).cumprod()
    
    # Create visualization
    plt.figure(figsize=(14, 6))
    plt.plot(top_cumulative_series.index, top_cumulative_series.values, label=top_sector_name, linewidth=2)
    plt.plot(other_cumulative_series.index, other_cumulative_series.values, label='Other Sectors', linewidth=2)
    plt.title('Cumulative Returns: Top Sector vs Other Sectors', fontsize=14, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Cumulative Return (Starting at 1.0)', fontsize=12)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Calculate final cumulative returns
    top_cumulative = (1 + top_sector_returns.dropna()).prod() - 1
    other_cumulative = (1 + other_sectors_returns.dropna()).prod() - 1
    
    print("\n" + "-"*80)
    print("KEY FINDINGS:")
    print("-"*80)
    print(f"Cumulative Return - {top_sector_name}: {top_cumulative*100:+.2f}%")
    print(f"Cumulative Return - Other Sectors: {other_cumulative*100:+.2f}%")
    print(f"Difference: {(top_cumulative - other_cumulative)*100:+.2f} percentage points")
    print(f"\n{top_sector_name} volatility: {top_sector_returns.std()*100:.4f}%")
    print(f"Other sectors volatility: {other_sectors_returns.std()*100:.4f}%")

In [6]:
# Function to analyze concentration trends over time
def analyze_concentration_trends():
    """
    Analyzes how sector concentration has changed over time
    """
    print("\n" + "="*80)
    print("SECTOR CONCENTRATION TRENDS OVER TIME")
    print("="*80 + "\n")
    
    # Get the top sector by market cap
    sector_marketcap = companies_df.groupby('Sector')['Marketcap'].sum().sort_values(ascending=False)
    top_sector = sector_marketcap.index[0]
    
    print(f"Analyzing concentration for: {top_sector}")
    print("-"*80 + "\n")
    
    # Get yearly snapshots for efficiency
    stocks_df['Year'] = stocks_df['Date'].dt.year
    yearly_dates = stocks_df.groupby('Year')['Date'].first().reset_index()
    
    concentration_data = []
    
    for _, row in yearly_dates.iterrows():
        date = row['Date']
        year = row['Year']
        
        # Get stock data for this date
        date_stocks = stocks_df[stocks_df['Date'] == date].copy()
        
        # Merge with sector info
        date_stocks = date_stocks.merge(companies_df[['Symbol', 'Sector']], on='Symbol')
        
        # Calculate value (using Close price as proxy)
        total_value = date_stocks['Close'].sum()
        top_sector_value = date_stocks[date_stocks['Sector'] == top_sector]['Close'].sum()
        
        if total_value > 0:
            sector_percentage = (top_sector_value / total_value) * 100
            concentration_data.append({
                'Year': year,
                f'{top_sector} %': round(sector_percentage, 2),
                'Other Sectors %': round(100 - sector_percentage, 2)
            })
    
    concentration_df = pd.DataFrame(concentration_data)
    
    print(f"{top_sector} Concentration by Year:")
    display(concentration_df)
    
    # Create visualization
    if len(concentration_df) > 0:
        plt.figure(figsize=(14, 6))
        col_name = f'{top_sector} %'
        plt.plot(concentration_df['Year'], concentration_df[col_name], marker='o', linewidth=2, markersize=8)
        plt.fill_between(concentration_df['Year'], concentration_df[col_name], alpha=0.3)
        plt.title(f'{top_sector} Concentration Over Time', fontsize=14, fontweight='bold')
        plt.xlabel('Year', fontsize=12)
        plt.ylabel(f'{top_sector} Percentage (%)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Calculate change over time
        start_pct = concentration_df[col_name].iloc[0]
        end_pct = concentration_df[col_name].iloc[-1]
        max_pct = concentration_df[col_name].max()
        min_pct = concentration_df[col_name].min()
        max_year = concentration_df.loc[concentration_df[col_name].idxmax(), 'Year']
        min_year = concentration_df.loc[concentration_df[col_name].idxmin(), 'Year']
        
        print("\n" + "-"*80)
        print("KEY FINDINGS:")
        print("-"*80)
        print(f"Starting percentage ({concentration_df['Year'].iloc[0]}): {start_pct:.2f}%")
        print(f"Ending percentage ({concentration_df['Year'].iloc[-1]}): {end_pct:.2f}%")
        print(f"Total change: {end_pct - start_pct:+.2f} percentage points")
        print(f"\nPeak concentration: {max_pct:.2f}% (in {max_year})")
        print(f"Lowest concentration: {min_pct:.2f}% (in {min_year})")
        print(f"Range: {max_pct - min_pct:.2f} percentage points")
    else:
        print("\nNo concentration data available.")

In [7]:
# Main interactive function
def explore_tech_concentration(analysis_type='Sector Distribution'):
    """
    Main interactive function to explore tech concentration in S&P 500
    """
    print("\n" + "="*80)
    print("THE GREAT TECH CONCENTRATION: An Analysis of the S&P 500")
    print("="*80 + "\n")
    
    if analysis_type == 'Sector Distribution':
        analyze_sector_distribution()
    elif analysis_type == 'Top Companies':
        analyze_top_companies(n=20)
    elif analysis_type == 'Performance Comparison':
        analyze_performance()
    elif analysis_type == 'Concentration Trends':
        analyze_concentration_trends()
    elif analysis_type == 'All Analyses':
        print("\n1. SECTOR DISTRIBUTION")
        print("-" * 80)
        analyze_sector_distribution()
        
        print("\n\n2. TOP COMPANIES ANALYSIS")
        print("-" * 80)
        analyze_top_companies(n=20)
        
        print("\n\n3. PERFORMANCE COMPARISON")
        print("-" * 80)
        analyze_performance()
        
        print("\n\n4. CONCENTRATION TRENDS")
        print("-" * 80)
        analyze_concentration_trends()

In [8]:
# Create the interactive widget
print("\n" + "="*80)
print("INTERACTIVE S&P 500 SECTOR CONCENTRATION EXPLORER")
print("="*80)
print("\nSelect an analysis type to explore sector concentration in the S&P 500:")
print()

interact(explore_tech_concentration, 
         analysis_type=widgets.Dropdown(
             options=['Sector Distribution', 'Top Companies', 'Performance Comparison', 
                      'Concentration Trends', 'All Analyses'],
             value='Sector Distribution',
             description='Analysis:',
             style={'description_width': 'initial'}
         ));


INTERACTIVE S&P 500 SECTOR CONCENTRATION EXPLORER

Select an analysis type to explore sector concentration in the S&P 500:



interactive(children=(Dropdown(description='Analysis:', options=('Sector Distribution', 'Top Companies', 'Perf…